In [3]:
from collections import deque
import random
import time
import tracemalloc

class Puzzle8:
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth

    def is_goal_state(self):
        return self.board == [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

    def get_possible_neighbors(self):
        neighbors = []
        zero_position = self.locate_zero_tile()
        valid_moves = self.get_available_moves(zero_position)

        for move in valid_moves:
            new_board = self.make_move(zero_position, move)
            neighbors.append(Puzzle8(new_board, parent=self, move=move, depth=self.depth + 1))
        return neighbors

    def locate_zero_tile(self):
        for row in range(3):
            for col in range(3):
                if self.board[row][col] == 0:
                    return (row, col)

    def get_available_moves(self, zero_pos):
        row, col = zero_pos
        moves = []
        if row > 0: moves.append((-1, 0))
        if row < 2: moves.append((1, 0))
        if col > 0: moves.append((0, -1))
        if col < 2: moves.append((0, 1))
        return moves

    def make_move(self, zero_pos, move):
        row, col = zero_pos
        row_offset, col_offset = move
        new_board = [row[:] for row in self.board]
        new_board[row][col], new_board[row + row_offset][col + col_offset] = new_board[row + row_offset][col + col_offset], new_board[row][col]
        return new_board

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.board])

def bfs_with_node_counting(initial_state):
    open_queue = deque([initial_state])
    closed_set = set()
    node_counter = 0

    while open_queue:
        current_state = open_queue.popleft()
        node_counter += 1

        if current_state.is_goal_state():
            return current_state, node_counter

        closed_set.add(str(current_state.board))

        for neighbor in current_state.get_possible_neighbors():
            if str(neighbor.board) not in closed_set:
                open_queue.append(neighbor)
                closed_set.add(str(neighbor.board))

    return None, node_counter

def retrieve_solution_path(goal_state):
    solution_path = []
    current_node = goal_state
    while current_node:
        solution_path.append(current_node)
        current_node = current_node.parent
    solution_path.reverse()
    return solution_path

def create_puzzle_at_depth(goal_board, depth):
    puzzle = Puzzle8(goal_board)
    for _ in range(depth):
        neighbors = puzzle.get_possible_neighbors()
        puzzle = random.choice(neighbors)
    return puzzle

def assess_performance_for_depth(depth):
    goal_board = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    puzzle = create_puzzle_at_depth(goal_board, depth)

    print(f"\nGenerated Puzzle at Depth {depth}:\n{puzzle}\n")

    tracemalloc.start()
    start_time = time.time()

    goal_state, nodes_visited = bfs_with_node_counting(puzzle)

    end_time = time.time()
    memory_used = tracemalloc.get_traced_memory()[1] / 1024
    tracemalloc.stop()

    return (end_time - start_time) * 1000, memory_used, nodes_visited

depth_levels = [1, 2, 3, 4, 5]
for depth in depth_levels:
    execution_time, memory_usage, nodes_counted = assess_performance_for_depth(depth)
    print(f"Depth {depth} -> Time: {execution_time:.2f} ms, Memory: {memory_usage:.2f} KB, Nodes Visited: {nodes_counted}")



Generated Puzzle at Depth 1:
1 2 3
4 5 6
7 0 8

Depth 1 -> Time: 1.60 ms, Memory: 5.01 KB, Nodes Visited: 4

Generated Puzzle at Depth 2:
1 2 3
4 0 6
7 5 8

Depth 2 -> Time: 0.91 ms, Memory: 7.48 KB, Nodes Visited: 9

Generated Puzzle at Depth 3:
1 2 3
0 5 6
4 7 8

Depth 3 -> Time: 1.36 ms, Memory: 12.02 KB, Nodes Visited: 13

Generated Puzzle at Depth 4:
1 2 3
4 5 6
0 7 8

Depth 4 -> Time: 0.53 ms, Memory: 4.79 KB, Nodes Visited: 7

Generated Puzzle at Depth 5:
1 0 3
4 2 5
7 8 6

Depth 5 -> Time: 1.01 ms, Memory: 11.26 KB, Nodes Visited: 15
